In [53]:
import gensim, glob, logging
from gensim.models.callbacks import CallbackAny2Vec

import sys
sys.path.append ("../modules")
from utils.w2vutils import EpochLogger

In [58]:
eqdocs_models = [gensim.models.Word2Vec.load(filename, mmap="r") for filename in sorted (glob.glob("/hg191/corpora/legaldata/sc-docs/*.model"), 
                                                                        key=lambda x:int(x.split("/")[-1].split(".")[0]))]

In [62]:
eqtime_models = [gensim.models.Word2Vec.load (filename, mmap="r") for filename in sorted (glob.glob ("/hg191/corpora/legaldata/sc-time/*.model"), key=len)]

Now check the synchronic validity of each model on some benchmark tasks and datasets.

In [66]:
!wget https://raw.githubusercontent.com/AdrienGuille/DistributionalSemantics/master/evaluation_data/wordsim353.tsv
!wget https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt

--2018-10-12 16:04:01--  https://raw.githubusercontent.com/AdrienGuille/DistributionalSemantics/master/evaluation_data/wordsim353.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.32.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7405 (7.2K) [text/plain]
Saving to: ‘wordsim353.tsv’

100%[======================================>] 7,405       --.-K/s   in 0s      

2018-10-12 16:04:01 (45.7 MB/s) - ‘wordsim353.tsv’ saved [7405/7405]

--2018-10-12 16:04:02--  https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.32.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603955 (590K) [text/plain]
Saving to: ‘questions-words.txt.1’

100%[===

In [85]:
model = eqtime_models[0]
similarities = model.wv.evaluate_word_pairs('wordsim353.tsv', dummy4unknown=False)
analogy_scores = model.wv.evaluate_word_analogies ("questions-words.txt", dummy4unknown=False)
print (similarities[0], similarities[1], analogy_scores[0])

/nethome/ssoni30/venvs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


(0.5395840044670418, 9.2791386480705e-28) SpearmanrResult(correlation=0.5570080301314163, pvalue=7.852727231162398e-30) 0.26904819863680624


In [89]:
correlations = list ()
analogy_accuracies = list ()
for i in range (1,6):
    similarities = eqtime_models[i].wv.evaluate_word_pairs('wordsim353.tsv', dummy4unknown=True)
    analogy_scores = eqtime_models[i].wv.evaluate_word_analogies ("questions-words.txt", dummy4unknown=True)
    print (i, similarities[0], similarities[1], analogy_scores[0])
    correlations.append (similarities[1].correlation)
    analogy_accuracies.append (analogy_scores[0])

/nethome/ssoni30/venvs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


1 (0.4088380994317934, 1.1778705445430818e-15) SpearmanrResult(correlation=0.4334708888875906, pvalue=1.3292362041002861e-17) 0.19274457634056488
2 (0.4147750578136015, 4.1326231017749983e-16) SpearmanrResult(correlation=0.4360834647415774, pvalue=8.082644031644541e-18) 0.19448424068767908
3 (0.4137560100481765, 4.953878155079962e-16) SpearmanrResult(correlation=0.4371147985338671, pvalue=6.6336345319732836e-18) 0.19535407286123618
4 (0.41041677754551165, 8.933555119628686e-16) SpearmanrResult(correlation=0.43634716424164294, pvalue=7.684980499987822e-18) 0.19315390912812117
5 (0.4113561522568702, 7.573112300672391e-16) SpearmanrResult(correlation=0.4355623410528807, pvalue=8.928885977662406e-18) 0.19223291035611953


In [84]:
import numpy as np
correlations = [0.4334708888875906, 0.4360834647415774, 0.4371147985338671, 0.43634716424164294, 0.4355623410528807]
analogy_accuracies = [0.19274457634056488, 0.19448424068767908, 0.19535407286123618, 0.19315390912812117, 0.19223291035611953]
print (np.mean (correlations), np.std(correlations))
print (np.mean (analogy_accuracies), np.std(analogy_accuracies))

0.43571573149151177 0.0012291599239472335
0.19359394187474416 0.0011539627801072797


Now let's evaluate on the models that have approx. equal number of documents in each of the splits.

In [90]:
correlations = list ()
analogy_accuracies = list ()
for i, model in enumerate (eqdocs_models):
    similarities = model.wv.evaluate_word_pairs('wordsim353.tsv', dummy4unknown=False)
    analogy_scores = model.wv.evaluate_word_analogies ("questions-words.txt", dummy4unknown=False)
    
    print (i, similarities[0], similarities[1].correlation, analogy_scores[0])
    
    correlations.append (similarities[1].correlation)
    analogy_accuracies.append (analogy_scores[0])

/nethome/ssoni30/venvs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0 (0.48535237306307105, 1.1403762301404674e-20) 0.4979876320664336 0.3473179502547198
1 (0.5210504539252125, 1.2153019805810918e-24) 0.5265069311258447 0.3253610108303249
2 (0.4992331775216031, 4.404449802530852e-23) 0.5190960757582349 0.3077999558076158
3 (0.5058419460263084, 6.0805499791123606e-24) 0.5225881507621538 0.3031134869609189
4 (0.5001200494375001, 4.804786515036026e-23) 0.5208830841944035 0.3015613910574876
5 (0.48202288459414044, 1.565624283556623e-21) 0.5014426857832559 0.3063122437024019
6 (0.4725359346534674, 1.1999366091480957e-20) 0.4879675646695335 0.28254488360980273
7 (0.5046689292816873, 6.91387642713424e-24) 0.5138656842999045 0.2897222030743779
8 (0.5155868606857807, 5.847387350807945e-25) 0.5325295677644264 0.29005270092226615
9 (0.5105085309038903, 1.7172409298959005e-24) 0.5265617110255899 0.2842859961849635
10 (0.5337414556945873, 7.179585612852385e-27) 0.5448029209401138 0.27216535956400373
11 (0.5332564370552508, 6.874646011316296e-27) 0.5526897528764858 

In [96]:
print (np.mean(correlations), np.std (correlations), np.min (correlations), np.max (correlations))
print (np.mean(analogy_accuracies), np.std (analogy_accuracies), np.min (analogy_accuracies), np.max (analogy_accuracies))

0.520576813438865 0.01784918059365494 0.4879675646695335 0.5526897528764858
0.29838960533884645 0.02115169689860722 0.27043808209727493 0.3473179502547198
